In [ ]:
# load data
from cot import Collection
coll = Collection("open_book_qa", load_pregenerated_cots=False)
coll = coll.select(split="test", number_samples=1)
coll

In [ ]:
# load local model

from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [13]:
"""Create a sequential langchain consisting of one for reasoning generation and the 
other for answer extraction. Inspired by https://js.langchain.com/docs/modules/chains/sequential_chain"""
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain


In [21]:
"""Reasoning chain"""

llm = local_llm

template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

{cot_trigger}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger"], template=template)
cot_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="cot")

In [22]:
"""answer extraction chain"""

extraction_template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

Cot: {cot_trigger}{cot}
{answer_extraction}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction"], template=extraction_template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="predicted_answer")

In [23]:
"""combination of generation and extraction chain"""

from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[cot_chain, answer_chain],
                                input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction"],
                                output_variables=["cot", "predicted_answer"],
                                verbose=True)

In [24]:
"""Create an input dictionary to feed the chain"""
input_dict = {'input_dict':
              {
                  'chain': overall_chain,
                  "instruction": "Be faithful and a little hopeful",
                  "cot_trigger": "Answer: Let's think step by step.",
                  "answer_extraction": "Therefore, among A through D, the answer is",
                  'model': "gpt-3.5-turbo",
                  'temperature': 0,
                  'max_tokens': 800
              }
              }

"""generate_extract_flexible is called in the dataloader where the collection resides."""    
coll.generate_extract_flexible(input_dict=input_dict)

Generating open_book_qa...


  0%|          | 0/1 [00:00<?, ?ex/s]



> Entering new SequentialChain chain...

> Finished chain.
{'id': 'open_book_qa_test_432', 'ref_id': '', 'question': 'How many times would someone change the page of a calendar in a year?', 'type': 'multiplechoice', 'choices': ['13', '12', '15', '14'], 'context': '', 'cot': ['one year is equal to 365 days'], 'answer': ['12'], 'generated_cot': [{'id': '8ac05ae7-b361-4a7d-940f-3098e4f9a18d', 'fragments_version': None, 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': 'C', 'answers': [{'id': '66e34967-e172-42df-8847-042cc7b10535', 'answer_extraction': 'Therefore, among A through D, the answer is', 'answer_extraction_template': '', 'answer_extraction_text': '', 'answer': 'A', 'answer_from_choices': '', 'correct_answer': None}], 'author': '', 'date': '2023/05/12 11:26:21', 'api_service': '', 'model': "{'name': 'gpt-3.5-turbo', 'temperature': 0, 'max_tokens': 800}", 'comment': 'genera

In [25]:
# Evaluate the answer given
coll.evaluate()

  0%|          | 0/1 [00:00<?, ?ex/s]

{'open_book_qa': {'test': {'accuracy': {'gpt-3.5-turbo': {"Be faithful and a little hopeful_Answer: Let's think step by step._Therefore, among A through D, the answer is": 0.0}}}}}